In [0]:
!pip install clarifai==9.9.3
!pip install pyspark


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [0]:
!pip install protobuf==4.24.2
!pip show protobuf
dbutils.library.restartPython() 
     


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Name: protobuf
Version: 4.24.2
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages
Requires: 
Required-by: clarifai-grpc, facets-overview, googleapis-common-protos, grpcio-status


# Writing ClarifaiPySpark SDK functions

### ClarifaiPySpark Dataset class

In [0]:
import json
import time
import uuid
from typing import Generator, List, Union 

import requests
from clarifai.client.app import App
from clarifai.client.dataset import Dataset
from clarifai.client.input import Inputs
from clarifai.client.user import User
from clarifai.errors import UserError
from clarifai_grpc.grpc.api.resources_pb2 import Input
from google.protobuf.json_format import MessageToJson
from google.protobuf.struct_pb2 import Struct
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import SparkSession



class Dataset(Dataset):
  """
  Dataset class provides information about dataset of the app
  and it inherits from the clarifai SDK Dataset class.
  """

  def __init__(self, user_id: str = "", app_id: str = "", dataset_id: str = ""):
    """Initializes the Dataset object.

    Args:
        user_id (str): The clarifai user ID of the user.
        app_id (str): Clarifai App ID.
        dataset_id (str): Dataset ID of the dataset inside the clarifai App.

    Example: TODO
    """
    self.user = User(user_id=user_id)
    self.app = App(app_id=app_id)
    #Inputs object - for listannotations
    #input_obj = User(user_id="user_id").app(app_id="app_id").inputs()
    self.user_id = user_id
    self.app_id = app_id
    self.dataset_id = dataset_id
    super().__init__(user_id=user_id, app_id=app_id, dataset_id=dataset_id)

  def upload_dataset_from_csv(self,
                              csv_path: str = "",
                              input_type: str = 'text',
                              csv_type: str = None,
                              labels: bool = True,
                              chunk_size: int = 128) -> None:
    """Uploads dataset to clarifai app from the csv file path.

    Args:
        csv_path (str): CSV file path of the dataset to be uploaded into clarifai App.
        input_type (str): Input type of the dataset whether (Image, text).
        csv_type (str): Type of the csv file contents(url, raw, filepath).
        labels (bool): Give True if labels column present in dataset else False.
        chunk_size (int): chunk size of parallel uploads of inputs and annotations.

    Example: TODO

    Note:
        CSV file supports 'inputid', 'input', 'concepts', 'metadata', 'geopoints' columns.
        All the data in the CSV should be in double quotes.
        metadata should be in single quotes format. Example: "{'key': 'value'}"
        geopoints should be in "long,lat" format.

    """
    ### TODO: Can input column names & extract them to convert to our csv format
    self.upload_from_csv(
        csv_path=csv_path,
        input_type=input_type,
        csv_type=csv_type,
        labels=labels,
        chunk_size=chunk_size)

  def upload_dataset_from_folder(self,
                                 folder_path: str,
                                 input_type: str,
                                 labels: bool = False,
                                 chunk_size: int = 128) -> None:
    """Uploads dataset from folder into clarifai app.

    Args:
        folder_path (str): folder path of the dataset to be uploaded into clarifai App.
        input_type (str): Input type of the dataset whether (Image, text).
        labels (bool): Give True if labels column present in dataset else False.
        chunk_size (int): chunk size of parallel uploads of inputs and annotations.

    Example: TODO

    Note:
        Can provide a volume or S3 path to the folder
        If label is true, then folder name is class name (label)
    """

    self.upload_from_folder(
        folder_path=folder_path, input_type=input_type, labels=labels, chunk_size=chunk_size)

  def get_inputs_from_dataframe(self,
                                dataframe,
                                input_type: str,
                                df_type: str,
                                dataset_id: str = None,
                                labels: str = True) -> List[Input]:
    input_protos = []
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)

    for row in dataframe.collect():
      if labels:
        labels_list = row["concepts"].split(',')
        labels = labels_list if len(row['concepts']) > 0 else None
      else:
        labels = None

      if 'metadata' in dataframe.columns:
        if row['metadata'] is not None and len(row['metadata']) > 0:
          metadata_str = row['metadata'].replace("'", '"')
          try:
            metadata_dict = json.loads(metadata_str)
          except json.decoder.JSONDecodeError:
            raise UserError("metadata column in CSV file should be a valid json")
          metadata = Struct()
          metadata.update(metadata_dict)
        else:
          metadata = None
      else:
        metadata = None

      if 'geopoints' in dataframe.columns:
        if row['geopoints'] is not None and len(row['geopoints']) > 0:
          geo_points = row['geopoints'].split(',')
          geo_points = [float(geo_point) for geo_point in geo_points]
          geo_info = geo_points if len(geo_points) == 2 else UserError(
              "geopoints column in CSV file should have longitude,latitude")
        else:
          geo_info = None
      else:
        geo_info = None

      input_id = row['inputid'] if 'inputid' in dataframe.columns else uuid.uuid4().hex
      text = row["input"] if input_type == 'text' else None
      image = row['input'] if input_type == 'image' else None
      video = row['input'] if input_type == 'video' else None
      audio = row['input'] if input_type == 'audio' else None

      if df_type == 'raw':
        input_protos.append(
            input_obj.get_text_input(
                input_id=input_id,
                raw_text=text,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))
      elif df_type == 'url':
        input_protos.append(
            input_obj.get_input_from_url(
                input_id=input_id,
                image_url=image,
                text_url=text,
                audio_url=audio,
                video_url=video,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))
      else:
        input_protos.append(
            input_obj.get_input_from_file(
                input_id=input_id,
                image_file=image,
                text_file=text,
                audio_file=audio,
                video_file=video,
                dataset_id=dataset_id,
                labels=labels,
                geo_info=geo_info))

    return input_protos

  def upload_dataset_from_dataframe(self,
                                    dataframe,
                                    input_type: str,
                                    df_type: str = None,
                                    labels: bool = True,
                                    chunk_size: int = 128) -> None:
    """Uploads dataset from a dataframe.
       Expected columns in the dataframe are inputid, input, concepts (optional), metadata (optional), geopoints (optional).

      Args:
          task (str): task type(text_clf, visual-classification, visual_detection, visual_segmentation, visual-captioning).
          split (str): split type(train, test, val).
          module_dir (str): path to the module directory.
          dataset_loader (str): name of the dataset loader.
          chunk_size (int): chunk size for concurrent upload of inputs and annotations.

      Example: TODO
    """

    if input_type not in ('image', 'text', 'video', 'audio'):
      raise UserError('Invalid input type, it should be image,text,audio or video')

    if df_type not in ('raw', 'url', 'file_path'):
      raise UserError('Invalid csv type, it should be raw, url or file_path')

    if df_type == 'raw' and input_type != 'text':
      raise UserError('Only text input type is supported for raw csv type')

    if not isinstance(dataframe, SparkDataFrame):
      raise UserError('dataframe should be a Spark DataFrame')

    chunk_size = min(128, chunk_size)
    input_obj = input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    input_protos = self.get_inputs_from_dataframe(
        dataframe=dataframe,
        df_type=df_type,
        input_type=input_type,
        dataset_id=self.dataset_id,
        labels=labels)
    return (input_obj._bulk_upload(inputs=input_protos, chunk_size=chunk_size))

  def upload_dataset_from_dataloader(self,
                                     task: str,
                                     split: str,
                                     module_dir: str = None,
                                     chunk_size: int = 128) -> None:
    """Uploads dataset using a dataloader function for custom formats.

    Args:
        task (str): task type(text_clf, visual-classification, visual_detection, visual_segmentation, visual-captioning).
        split (str): split type(train, test, val).
        module_dir (str): path to the module directory.
        chunk_size (int): chunk size for concurrent upload of inputs and annotations.

    Example: TODO
    """
    self.upload_dataset(task=task, split=split, module_dir=module_dir, chunk_size=chunk_size)

  def upload_dataset_from_table(self,
                                table_path: str,
                                input_type: str,
                                table_type: str,
                                labels: bool,
                                chunk_size: int = 128) -> None:
    """upload dataset to clarifai app from spark tables.

    Args:
        table_path (str): path of the table to be uploaded.
        task (str):
        split (str):
        input_type (str): Input type of the dataset whether (Image, text).
        table_type (str): Type of the table contents (url, raw, filepath).
        labels (bool): Give True if labels column present in dataset else False.
        module_dir (str): path to the module directory.
        dataset_loader (str): name of the dataset loader.
        chunk_size (int): chunk size for concurrent upload of inputs and annotations.
    Note:
        Accepted csv format - input, label
        TODO: dataframe dataloader template
        TODO: Can input column names & extreact them to convert to our csv format
    """
    spark = SparkSession.builder.appName('Clarifai-spark').getOrCreate()
    tempdf = spark.read.format("delta").load(table_path)
    self.upload_from_dataframe(
        dataframe=tempdf,
        input_type=input_type,
        df_type=table_type,
        labels=labels,
        chunk_size=chunk_size)

  def list_inputs(self, per_page: int = None, input_type: str = None) -> Generator[Input, None, None]:
    """Lists all the inputs from the app.

    Args:
        per_page (str): No of response of inputs per page.
        input_type (str): Input type that needs to be displayed (text,image)
        TODO: Do we need input_type ?, since in our case it is image, so probably we can go with default value of "image".

    Examples:
        TODO

    Returns:
        list of inputs.
        """
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    return input_obj.list_inputs(
        dataset_id=self.dataset_id, input_type=input_type, per_page=per_page)

  def list_annotations(self, per_page: int = None, input_type: str = None):
    """Lists all the annotations for the inputs in the dataset of a clarifai app.

    Args:
        per_page (str): No of response of inputs per page.
        input_type (str): Input type that needs to be displayed (text,image)
        TODO: Do we need input_type ?, since in our case it is image, so probably we can go with default value of "image".

    Examples:
        TODO

    Returns:
        list of annotations.
    """
    ### input_ids: list of input_ids for which user wants annotations
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    all_inputs = list(
        input_obj.list_inputs(
            dataset_id=self.dataset_id, input_type=input_type, per_page=per_page))
    return input_obj.list_annotations(batch_input=all_inputs)

  def export_annotations_to_dataframe(self):
    """Export all the annotations from clarifai App to spark dataframe.

    Examples:
        TODO

    Returns:
        spark dataframe with annotations"""

    annotation_list = []
    spark = SparkSession.builder.appName('Clarifai-spark').getOrCreate()
    input_obj = Inputs(user_id=self.user_id, app_id=self.app_id)
    all_inputs = list(input_obj.list_inputs(dataset_id=self.dataset_id))
    response = list(input_obj.list_annotations(batch_input=all_inputs))
    for an in response:
      temp = {}
      temp['annotation'] = json.loads(MessageToJson(an.data))
      if not temp['annotation'] or temp['annotation'] == '{}' or temp['annotation'] == {}:
        continue
      temp['id'] = an.id
      temp['user_id'] = an.user_id
      temp['input_id'] = an.input_id
      try:
        created_at = float(f"{an.created_at.seconds}.{an.created_at.nanos}")
        temp['created_at'] = time.strftime('%m/%d/% %H:%M:%5', time.gmtime(created_at))
        modified_at = float(f"{an.modified_at.seconds}.{an.modified_at.nanos}")
        temp['modified_at'] = time.strftime('%m/%d/% %H:%M:%5', time.gmtime(modified_at))
      except:
        temp['created_at'] = float(f"{an.created_at.seconds}.{an.created_at.nanos}")
        temp['modified_at'] = float(f"{an.modified_at.seconds}.{an.modified_at.nanos}")
      annotation_list.append(temp)
    return spark.createDataFrame(annotation_list)

  def export_images_to_volume(self, path, input_response):
    for resp in input_response:
      imgid = resp.id
      ext = resp.data.image.image_info.format
      url = resp.data.image.url
      img_name = path + '/' + imgid + '.' + ext.lower()
      headers = {"Authorization": self.metadata[0][1]}
      response = requests.get(url, headers=headers)
      with open(img_name, "wb") as f:
        f.write(response.content)

  def export_text_to_volume(self, path, input_response):
    for resp in input_response:
      textid = resp.id
      url = resp.data.text.url
      file_name = path + '/' + textid + '.txt'
      enc = resp.data.text.text_info.encoding
      headers = {"Authorization": self.metadata[0][1]}
      response = requests.get(url, headers=headers)
      with open(file_name, "a", encoding=enc) as f:
        f.write(response.content.decode())


2023-10-23 09:04:30 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=574375;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=783822;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### ClarifaiPySpark Client class

In [0]:
from clarifai.client.app import App
from clarifai.client.base import BaseClient
from clarifai.client.user import User

# from clarifaipyspark.dataset import Dataset


class ClarifaiPySpark(BaseClient):
  """
  ClarifaiPySpark inherits the BaseClient class from the clarifai SDK and it initializes the client.
  """

  def __init__(self, user_id: str = "", app_id: str = ""):
    """Initializes clarifai client object.

    Args:
      - user_id (str): A user ID for authentication.
      - app_id (str): An app ID for the application to interact with.
    """

    self.user = User(user_id=user_id)
    self.app = App(app_id=app_id)
    self.user_id = user_id
    self.app_id = app_id
    super().__init__(user_id=user_id, app_id=app_id)

  def dataset(self, dataset_id):
    """Initializes the dataset method with dataset_id.

    Args:
      dataset_id: The dataset_id within the user app.

    Returns:
      Dataset object for the dataset_id.
    """

    try:
      self.app.dataset(dataset_id=dataset_id)
    except:
      print("Creating a new dataset")
      self.app.create_dataset(dataset_id=dataset_id)

    return Dataset(dataset_id=dataset_id, user_id=self.user_id, app_id=self.app_id)


2023-10-23 09:04:33 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=85542;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=669343;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

# Testing ClarifaiPySpark SDK

### Setting Env variables

In [0]:
import os

os.environ['CLARIFAI_PAT'] = ''

2023-10-23 09:04:36 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=489096;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=194774;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Creating ClarifaiPyspark object & creating/fetching image dataset from app

In [0]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_img')

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-10-23 09:04:40 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=498902;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=634484;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from image dataset

In [0]:
my_inputs = list(dataset_obj.list_inputs())
my_inputs

2023-10-23 09:07:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=653055;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=627783;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "dataset1-train-3647386"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_img/inputs/image/1249d3d8c8157d1b2dcdb2561175374a"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester_img/inputs/image/1249d3d8c8157d1b2dcdb2561175374a"
       sizes: "orig"
       sizes: "tiny"
       sizes: "small"
       sizes: "large"
       crossorigin: "use-credentials"
     }
     image_info {
       width: 512
       height: 288
       format: "JPEG"
       color_mode: "YUV"
     }
   }
   concepts {
     id: "id-hamburger"
     name: "hamburger"
     value: 1
     app_id: "databricks_tester_img"
   }
 }
 created_at {
   seconds: 1698052012
   nanos: 477555000
 }
 modified_at {
   seconds: 1698052013
   nanos: 977120000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "dataset1-train-3520891"
 data {
   image {
     url: "https://data.clarifai.com/orig/u

### Upload images from volume folder to Clarifai dataset

In [0]:
dataset_obj.upload_dataset_from_folder(folder_path='/Volumes/mansi_test/default/cat', input_type='image', labels=True)

2023-10-23 08:27:37 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=55296;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=786623;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:05<00:00,  5.36s/it]


### Export image annotations to spark dataframe

In [0]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df

2023-10-23 08:30:22 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=777205;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=86519;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

DataFrame[annotation: map<string,array<map<string,string>>>, created_at: string, id: string, input_id: string, modified_at: string, user_id: string]

### Write annotations dataframe to volume as delta table

In [0]:
annot_df.write.format("delta").mode("overwrite").save("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable1") 


2023-10-23 08:30:51 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=660796;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=177253;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load annotations delta table from volume

In [0]:
df_delta = spark.read.format("delta").load("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable1")
df_delta.show()

2023-10-23 08:31:11 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=857207;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=734426;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+--------------------+--------+-------------------+-------+
|          annotation|         created_at|                  id|input_id|        modified_at|user_id|
+--------------------+-------------------+--------------------+--------+-------------------+-------+
|{concepts -> [{na...|10/23/% 08:27:   %5|da7eb3483a654a42a...|      c5|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|f65d75b3446b40ffa...|     c11|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|8fd99dc05eef46849...|      c1|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|9426763104ea42fbb...|      c7|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|d93bb8387f6743b8a...|      c8|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|aed44152eaad4fbcb...|      c6|10/23/% 08:27:   %5|mansi_k|
|{concepts -> [{na...|10/23/% 08:27:   %5|b3847d13486f4f118...|      c2|10/23/% 08:27:   %5

### Loop through delta table rows & columns

In [0]:
for row in df_delta.collect()[:4]:
    print(row['id'])
    print(row['input_id'])
    print(row['user_id'])
    print(row['created_at'])
    print(row['input_id'])
    print(row['annotation'])
    print("########################\n")

2023-10-23 08:31:26 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=698390;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=992550;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

da7eb3483a654a42ace1ebac76a69fdc
c5
mansi_k
10/23/% 08:27:   %5
c5
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

f65d75b3446b40ffa012735a31dcbb62
c11
mansi_k
10/23/% 08:27:   %5
c11
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

8fd99dc05eef468492623153531700c9
c1
mansi_k
10/23/% 08:27:   %5
c1
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################

9426763104ea42fbb44ff73677fd3806
c7
mansi_k
10/23/% 08:27:   %5
c7
{'concepts': [{'name': 'cat', 'id': 'id-cat', 'value': '1.0', 'appId': 'databricks_tester_img'}]}
########################



### Testing export_images_to_volume()

In [0]:
dataset_obj.export_images_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs[:5])

2023-10-23 08:31:44 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=244234;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=125648;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load image csv file & upload text data into app using dataframe

In [0]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/img_data2.csv")
df.show()

2023-10-23 09:02:37 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=783896;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=96789;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|  img11|https://img.freep...|     dog|
|  img21|https://images.un...|    duck|
|  img31|https://mymodernm...|    duck|
+-------+--------------------+--------+



In [0]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='image', df_type='url', labels=True)

2023-10-23 09:02:44 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=315145;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=593548;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-23 09:02:45 WARNING  WARNING:clarifai.client.input:code: INPUT_DUPLICATE                       ]8;id=742428;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py\input.py]8;;\:]8;id=293937;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py#631\631]8;;\
                             description: "Duplicate URL in your application. Check the documentation              
                             to allow duplications."                                                               
                             details: "The input URL is duplicate."                                                
                                                                                                                   

Uploading inputs: 100%|██████████| 1/1 [00:00<00:00,  4.48it/s]


### Using a custom dataloader to upload images from Volume

In [0]:
dataset_obj.upload_dataset_from_dataloader(task="visual-classification", split="train", module_dir="/Volumes/mansi_test/default/test_vol1/food-101")

2023-10-23 09:06:51 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=977950;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=507837;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading Dataset: 100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


### Creating ClarifaiPyspark object & creating/fetching text dataset from app

In [0]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_txt')

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-10-23 08:33:36 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=643398;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=913909;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Creating a new dataset


                    INFO     INFO:clarifai.client.app:                                                   ]8;id=28864;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/app.py\app.py]8;;\:]8;id=868935;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/app.py#291\291]8;;\
                             Dataset created                                                                       
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "ad013b11e5cf2894d78d3a5dfc368202"                                            
                                                                                                                   

### Fetching current inputs from text dataset

In [0]:
my_inputs = dataset_obj.list_inputs()

my_inputs

2023-10-23 08:34:24 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=537671;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=861851;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

<generator object Inputs.list_inputs at 0x7f92a88c6570>

### Testing export_text_to_volume()

Q- Create folders acc to concept name?   
Q- Save text as csv?

In [0]:
dataset_obj.export_text_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs)

2023-10-23 08:34:45 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=654716;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=492410;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Upload text from volume csv to Clarifai dataset

In [0]:
dataset_obj.upload_dataset_from_csv(csv_path='/Volumes/mansi_test/default/test_vol1/emotions_data1.csv', input_type='text', labels=True, csv_type='raw')

2023-10-23 08:38:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=990114;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=279494;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:20<00:00, 20.56s/it]


### Load test csv file & upload text data into app using dataframe

In [0]:
spark = SparkSession.builder.appName("Clarifai-spark").getOrCreate()
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/emotions_data1.csv")
df.show()

2023-10-23 08:38:54 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=298316;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=912974;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|    t11|   I am feeling good|   happy|
|    t21|This is a guava tree| neutral|
|    t31|This is a saddeni...|     sad|
+-------+--------------------+--------+



In [0]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='text', df_type='raw', labels=True)

2023-10-23 08:39:47 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=241980;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=257321;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-23 08:39:48 WARNING  WARNING:clarifai.client.input:code: INPUT_INVALID_ARGUMENT                ]8;id=620274;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py\input.py]8;;\:]8;id=60565;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py#631\631]8;;\
                             description: "Input invalid argument"                                                 
                             details: "Input has a duplicate ID."                                                  
                                                                                                                   

Uploading inputs: 100%|██████████| 1/1 [00:00<00:00,  5.45it/s]
